In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [2]:
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  8% |  0% |


In [20]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, sampler
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import random
import os
from torchmetrics import F1Score

In [4]:
random_seed = 12
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [5]:
train_dir_path = '/opt/ml/input/data/train/'
train_image_path = '/opt/ml/input/data/train/images/'

dt_train = pd.read_csv(train_dir_path+'train.csv')
dt_train

,id,gender,race,age,path
0,1,female,Asian,45,000001_female_Asian_45
1,2,female,Asian,52,000002_female_Asian_52
2,4,male,Asian,54,000004_male_Asian_54
3,5,female,Asian,58,000005_female_Asian_58
4,6,female,Asian,59,000006_female_Asian_59
...,...,...,...,...,...
2695,6954,male,Asian,19,006954_male_Asian_19
2696,6955,male,Asian,19,006955_male_Asian_19
2697,6956,male,Asian,19,006956_male_Asian_19
2698,6957,male,Asian,20,006957_male_Asian_20


In [6]:
def get_age_range(age):
    if age < 30:
        return 0
    elif 30 <= age < 60:
        return 1
    else:
        return 2

In [7]:
dt_train['age_range'] = dt_train['age'].apply(lambda x : get_age_range(x))

In [8]:
over_sixty = dt_train.loc[dt_train['age_range']==2,:]
path1 = []
path2 = []
for i in range(len(over_sixty)):
    path1.append(over_sixty.iloc[i]['path'] + '_1')
    path2.append(over_sixty.iloc[i]['path'] + '_2')
over_sixty_1 = pd.DataFrame({'id':over_sixty['id'], 'gender':over_sixty['gender'], 'race':over_sixty['race'], 'path':path1, 'age_range':over_sixty['age_range']})
over_sixty_2 = pd.DataFrame({'id':over_sixty['id'], 'gender':over_sixty['gender'], 'race':over_sixty['race'], 'path':path2, 'age_range':over_sixty['age_range']})
dt_train = dt_train.append(over_sixty_1, ignore_index=True)
dt_train = dt_train.append(over_sixty_2, ignore_index=True)
dt_train

,id,gender,race,age,path,age_range
0,1,female,Asian,45.0,000001_female_Asian_45,1
1,2,female,Asian,52.0,000002_female_Asian_52,1
2,4,male,Asian,54.0,000004_male_Asian_54,1
3,5,female,Asian,58.0,000005_female_Asian_58,1
4,6,female,Asian,59.0,000006_female_Asian_59,1
...,...,...,...,...,...,...
3113,5453,female,Asian,NaN,005453_female_Asian_60_2,2
3114,5459,male,Asian,NaN,005459_male_Asian_60_2,2
3115,5461,female,Asian,NaN,005461_female_Asian_60_2,2
3116,5504,female,Asian,NaN,005504_female_Asian_60_2,2


In [9]:
train_idx, valid_idx = train_test_split(np.arange(len(dt_train)),
                                       test_size=0.2,
                                       shuffle=True,
                                       stratify=dt_train['age_range'])

In [10]:
train_image = []
train_label = []

for idx in train_idx:
    path = dt_train.iloc[idx]['path']
    for file_name in [i for i in os.listdir(train_image_path+path) if i[0] != '.']:
        train_image.append(train_image_path+path+'/'+file_name)
        train_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0]))            

In [11]:
valid_image = []
valid_label = []

for idx in valid_idx:
    path = dt_train.iloc[idx]['path']
    for file_name in [i for i in os.listdir(train_image_path+path) if i[0] != '.']:
        valid_image.append(train_image_path+path+'/'+file_name)
        valid_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0]))                

In [12]:
def onehot_enc(x):
    def gender(i):
        if i == 'male':
            return 0
        elif i == 'female':
            return 3
    def age(j):
        j = int(j)
        if j < 30:
            return 0
        elif j >= 30 and j < 60:
            return 1
        elif j >= 60:
            return 2
    def mask(k):
        if k == 'normal':
            return 12
        elif 'incorrect' in k:
            return 6
        else:
            return 0
    return gender(x[0]) + age(x[1]) + mask(x[2])

In [13]:
train_data = pd.Series(train_image)
train_label = pd.Series(train_label)

valid_data = pd.Series(valid_image)
valid_label = pd.Series(valid_label)

In [14]:
class Dataset_Mask(Dataset):
    def __init__(self, data, label, encoding=True, midcrop=True, transform=None):
        self.encoding = encoding
        self.midcrop = midcrop
        self.data = data
        self.label = label
        self.transform = transform
        
        if encoding:
            self.label = self.label.apply(onehot_enc)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        X = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        y = self.label[idx]
        
        if self.midcrop:
            X = X[70:420, 17:367]
        
        if self.transform:
            return self.transform(X), y
        return X, y

In [15]:
mask_train_set = Dataset_Mask(data=train_data, label=train_label, transform = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.RandomHorizontalFlip(), 
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                            ]))
mask_val_set = Dataset_Mask(data=valid_data, label=valid_label, transform = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                            ]))

In [17]:
def make_class_weights(labels, class_num):
    print("Labels shape:\n", labels.shape)
    print("Given labels:\n", labels)

    labels = labels.apply(lambda x : x % class_num)

    labels = np.array(labels)
    class_weights = np.zeros_like(labels) 
    
    _, counts = np.unique(labels, return_counts=True)
    # 각 class가 몇 번 등장하는지 count

    print("Labels:\n", labels)
    print("Label count:\n", counts)

    for cls in range(class_num):
        class_weights = np.where(labels == cls, 1/counts[cls], class_weights)
        # label이 class에 해당하면 count의 역수 적용
    return class_weights

class_num = 18
age_class_num = 3

In [18]:
# class_weights = make_class_weights(mask_train_set.label, class_num)
class_weights = make_class_weights(mask_train_set.label, age_class_num)

print("Class weights: ", class_weights)
print("Length: ", len(class_weights))

Labels shape:
 (17458,)
Given labels:
 0         5
1         5
2         5
3        11
4         5
         ..
17453     3
17454     9
17455     3
17456     3
17457    15
Length: 17458, dtype: int64
Labels:
 [2 2 2 ... 0 0 0]
Label count:
 [7175 6769 3514]
Class weights:  [0.00028458 0.00028458 0.00028458 ... 0.00013937 0.00013937 0.00013937]
Length:  17458


In [21]:
sampler = sampler.WeightedRandomSampler(weights=class_weights, num_samples=len(class_weights))

In [22]:
print(f'training data size : {len(mask_train_set)}')
print(f'validation data size : {len(mask_val_set)}')

training data size : 17458
validation data size : 4368


In [23]:
batch_size = 64

train_dataloader_mask = DataLoader(dataset = mask_train_set, batch_size=batch_size, sampler=sampler, num_workers=2)
val_dataloader_mask = DataLoader(dataset = mask_val_set, batch_size=batch_size, num_workers=2)

In [24]:
model = torchvision.models.resnet50(pretrained=True)
print('필요 입력 채널 개수', model.conv1.weight.shape[1])
print('네트워크 출력 채널 개수', model.fc.weight.shape[0])
print(model)

/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


필요 입력 채널 개수 3
네트워크 출력 채널 개수 1000
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256

In [25]:
import math
class_num = 18
model.fc = nn.Linear(in_features=2048, out_features=class_num, bias=True)
nn.init.xavier_uniform_(model.fc.weight)
stdv = 1. / math.sqrt(model.fc.weight.size(1))
model.fc.bias.data.uniform_(-stdv, stdv)

print('필요 입력 채널 개수', model.conv1.weight.shape[1])
print('네트워크 출력 채널 개수', model.fc.weight.shape[0])

필요 입력 채널 개수 3
네트워크 출력 채널 개수 18


In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"using {device}")

model.to(device)

LEARNING_RATE = 0.0001
NUM_EPOCH = 50

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

using cuda:0


In [28]:
best_val_acc = 0
best_val_loss = np.inf
patience = 10
cur_count = 0
accumulation_steps = 4

f1 = F1Score(num_classes=class_num, average='macro').to(device)
best_f1_score = 0

for epoch in range(NUM_EPOCH):
    model.train()
    loss_value = 0
    matches = 0
    for idx, train_batch in enumerate(tqdm(train_dataloader_mask)):
        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outs = model(inputs)
        preds = torch.argmax(outs, dim=-1)
        loss = criterion(outs, labels)
        
        loss.backward()
        
        if (idx+1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            
        if epoch % 10 == 0:
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch' : epoch
                }, f'/opt/ml/checkpoint/res50_aug/checkpoint_ep_{epoch+1}.tar')
        
        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        
    train_loss = loss_value / len(train_dataloader_mask)
    train_acc = matches / len(mask_train_set)
        
    print(f"epoch[{epoch+1}/{NUM_EPOCH}] training loss {train_loss:.5f}, training accuracy {train_acc:.5f}")
        
    with torch.no_grad():
        model.eval()
        val_loss_items = []
        val_acc_items = []
        f1_score = 0
        for val_batch in tqdm(val_dataloader_mask):
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            
            loss_item = criterion(outs, labels).item()
            acc_item = (labels==preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            
        val_loss = np.sum(val_loss_items) / len(val_dataloader_mask)
        val_acc = np.sum(val_acc_items) / len(mask_val_set)

        f1_score /= len(val_dataloader_mask)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch' : epoch
                }, f'/opt/ml/checkpoint/res50_aug/best_checkpoint.tar')
            print('checkpoint saved!')
            cur_count = 0
        else:
            cur_count += 1
            if cur_count >= patience:
                print("Early Stopping!")
                break
            
        if f1_score > best_f1_score:
            best_f1_score = f1_score
            
            
        print(f"[val] acc : {val_acc:.5f}, loss : {val_loss:.5f}, f1 score: {f1_score:.5f}")
        print(f"best acc : {best_val_acc:.5f}, best loss : {best_val_loss:.5f}, best f1 : {best_f1_score:.5f}")


epoch[1/50] training loss 0.54050, training accuracy 0.83721



checkpoint saved!
[val] acc : 0.91667, loss : 0.23888, f1 score: 0.00000
best acc : 0.91667, best loss : 0.23888, best f1 : 0.00000



epoch[2/50] training loss 0.08818, training accuracy 0.97503



checkpoint saved!
[val] acc : 0.92766, loss : 0.23170, f1 score: 0.00000
best acc : 0.92766, best loss : 0.23170, best f1 : 0.00000



epoch[3/50] training loss 0.03428, training accuracy 0.99118


...


[val] acc : 0.94940, loss : 0.23232, f1 score: 0.00000
best acc : 0.95421, best loss : 0.20172, best f1 : 0.00000



epoch[21/50] training loss 0.00643, training accuracy 0.99828



[val] acc : 0.91552, loss : 0.38373, f1 score: 0.00000
best acc : 0.95421, best loss : 0.20172, best f1 : 0.00000



epoch[22/50] training loss 0.01082, training accuracy 0.99651



Early Stopping!


In [29]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir_path = '/opt/ml/input/data/eval/'
test_image_path = '/opt/ml/input/data/eval/images/'

checkpoint = torch.load('/opt/ml/checkpoint/res50_aug/best_checkpoint.tar')
model.load_state_dict(checkpoint['model_state_dict'])

submission = pd.read_csv(test_dir_path+'info.csv')
submission.head()

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0


In [30]:
image_paths = [os.path.join(test_image_path, img_id) for img_id in submission.ImageID]
test_image = pd.Series(image_paths)

In [31]:
class Test_Dataset(Dataset):
    def __init__(self, midcrop=True, transform=None):
        self.midcrop = midcrop
        self.data = test_image
        self.transform = transform
        
    def __len__(self):
        return len(test_image)
    
    def __getitem__(self, idx):
        img = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        
        if self.midcrop:
            img = img[70:420, 17:367]
            
        if self.transform:
            img = self.transform(img)
            
        return img

In [32]:
dataset = Test_Dataset(transform = transforms.Compose([
                            transforms.ToPILImage(),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                        ]))

loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = model.to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir_path, 'submission_res50_aug.csv'), index=False)
print('test inference is done!')

test inference is done!
